<img src='images/gdd-logo.png' width='300px' align='right' style="padding: 15px">

# Transformers

In the previous notebook, you developed a baseline model. However, we made you ignore any feature that contained text and any row that contained missing data.

In this notebook, you shall use sklearn Transformers to perform data preprocessing and investigate how it can aid model performance.

- [Baseline model](#baseline)
- [Sklearn Objects](#sklearn)
- [Sklearn Transformers]()
    - [Treating categorical columns](#cat)
    - [Using `ColumnTransformer` for a subset of columns](#ct)
    - [Using `SimpleImputer` to treat missing values](#impute)
- [The sklearn `Pipeline`](#pipeline)
- [Conclusion and next steps](#conc)

<a id=baseline></a>

## Recreating the Baseline Model

Let's make sure to have access to the baseline model during this notebook so you can compare performance.

Import the relevant packages and modules:

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier

In [3]:
stroke = pd.read_csv('data/stroke.csv').rename(columns=str.lower)
stroke.head()

,id,address,gender,age,who,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,"Vanessaside, VT 79393",Male,67.0,Man,0,1,1,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,APO AA 41945,Female,NaN,Woman,0,0,1,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,"North Jenniferfurt, VT 88077",Male,80.0,Man,0,1,1,Private,Rural,105.92,32.5,never smoked,1
3,60182,"Rodriguezside, LA 93111",Female,49.0,Woman,0,0,1,Private,Urban,171.23,34.4,smokes,1
4,1665,"Ivanburgh, AZ 80818",Female,79.0,Woman,1,0,1,Self-employed,Rural,174.12,24.0,never smoked,1


Create X and y:

In [4]:
categorical_cols = ['work_type', 'smoking_status', 'who', 'gender', 'residence_type']
missing_cols = ['bmi', 'age']
drop_cols = ['id','address']

target = 'stroke'

# Function to split data into X and y
def create_Xy(df, drop_cols, target_col):
    df = df.drop(columns=drop_cols)
    return (
        df.drop(columns=target_col),
        df[target_col]
    )

X_baseline, y = stroke.pipe(create_Xy, 
                        drop_cols=drop_cols
                        + categorical_cols
                        + missing_cols, 
                        target_col='stroke')

Split the data in training and test set:

In [5]:
train_test_params = {
    'test_size':0.25, 
    'random_state':42, 
    'stratify':y
}

X_baseline_train, X_baseline_test, y_train, y_test = train_test_split(
                                                                      X_baseline, 
                                                                      y, 
                                                                      **train_test_params)

Recreate and train the base model (Decision Tree Classifier):

In [6]:
# Step 1: Import model
from sklearn.tree import DecisionTreeClassifier

# Step 2: Instantiate model and set parameters
base_model = DecisionTreeClassifier(max_depth=3, 
                                    class_weight='balanced',
                                    random_state=42)

# Step 3: Train model
base_model.fit(X_baseline_train, y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=3, random_state=42)

In [7]:
# Step 4: Evaluate model
y_baseline_train_probs = base_model.predict_proba(X_baseline_train)[:,1]
y_baseline_test_probs = base_model.predict_proba(X_baseline_test)[:,1]

print(f'AUC: {roc_auc_score(y_train, y_baseline_train_probs), roc_auc_score(y_test, y_baseline_test_probs)}')

AUC: (0.739189613576196, 0.7298851337011885)


In this notebook, you will try to improve on the ROC-AUC scores achieved with your simple models through the use of sklearn **Transformers**.

<a id=sklearn></a>

## Sklearn Objects: Estimators, Predictors, Models and Transformers

Sklearn is built up of different types of [Objects](https://scikit-learn.org/stable/developers/develop.html). 

- An **Estimator** which implements a fit method to learn from data. 
- A **Predictor** makes predictions using the `.predict()` method.
- A **Model** can give a goodness of fit measure or a likelihood of unseen data using a `.score()` method.
- A **Transformer** can be used for filtering or modifying the data with the `.transform()` and `.fit_transform()` methods.

The Decision Tree algorithm is an example of an **Estimator** since it will use the `.fit()` method to apply the decision tree algorithm to some given data. Once fitted it becomes both a **Predictor** and a **Model** since, once fitted, it can make predictions and supply a measure of goodness of fit.

The great thing about sklearn is that all model algorithms follow this pattern. 

### Transformers

You will now use **Transformers** to *transform* the data (who would have guessed). 

Transformers help you:
- deal with missing values (e.g., by imputation)
- deal with categorical and string features.
- create new features from existing ones (e.g., by adding polynomial features and interactions)
- and much more...


Let's recreate X and y without dropping the categorical features or features with missing values.

In [8]:
drop_cols = ['id','address']

X, y = stroke.pipe(create_Xy, 
                   drop_cols=drop_cols, 
                   target_col='stroke')

X_train, X_test, y_train, y_test = train_test_split(X, y, **train_test_params)

In [9]:
X_train.head()

,gender,age,who,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status
3586,Female,72.0,Woman,1,1,1,Private,Urban,198.32,31.3,formerly smoked
1470,Male,18.0,Man,0,0,0,Private,Urban,83.02,40.4,Unknown
4431,Female,13.0,Child,0,0,0,children,Rural,146.10,22.8,never smoked
2397,Male,NaN,Child,0,0,0,children,Urban,115.12,21.1,Unknown
717,Female,13.0,Child,0,0,0,children,Rural,63.26,19.5,Unknown


<a id=cat></a>

### Treating categorical data

The data now contains categorical and string features.

We will have to use **One-hot Encoding** or **Ordinal Encoding** to transform the categories into numerical features.

####  <mark>**Exercise**</mark> 

1. Which features do you consider categorical?
2. Sklearn implements the [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) and [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) to deal with categorical data. Find out what they do.
3. For which ones would you use one-hot encoding and for which ones ordinal encoding?

In [10]:
# Find columns of DType "object"
stroke.select_dtypes('O').columns

Index(['address', 'gender', 'who', 'work_type', 'residence_type',
       'smoking_status'],
      dtype='object')

In [11]:
stroke['who'].value_counts()

who
Woman    2576
Man      1677
Child     856
Name: count, dtype: int64

<details>
  <summary><span style="color:blue">Show solution</span></summary>
    
  You can use `stroke.select_dtypes('O').nunique()` to see how many unique categories each feature contains.
  
  None of the columns have an ordinal relationship (`smoking_status` may be a candidate but where would `unknown` fit?).
  We should therfore use `OneHotEncoder`.

</details>

<a id=ct></a>

### Using `ColumnTransformer` to select columns

Of course, we only want to transform the categorical columns and leave the numerical columns as they are.

What happens when you use `OneHotEncoder` on the entire dataset?

In [12]:
print(X_train.shape)
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder()
onehot.fit_transform(X_train).shape

(3831, 11)


(3831, 3672)

Let's look at the names of the outputted columns. 

*Note: On sklearn 1.0+ the method is `get_feature_names_out()`. For earlier versions use `get_feature_names()`.*

In [13]:
onehot.get_feature_names_out()[:20]

array(['gender_Female', 'gender_Male', 'age_0.16', 'age_0.24', 'age_0.32',
       'age_0.4', 'age_0.48', 'age_0.56', 'age_0.64', 'age_0.72',
       'age_0.8', 'age_0.88', 'age_1.0', 'age_1.08', 'age_1.16',
       'age_1.24', 'age_1.32', 'age_1.4', 'age_1.48', 'age_1.56'],
      dtype=object)

<mark>**Question:**</mark> What happened?

<details>
  <summary><span style="color:blue">Show solution</span></summary>
    
  You have created a new column for each new unique value of each column, regardless of whether the data was categorical or not. Instead you want to apply the `OneHotEncoder` and `OrdinalEncoder` only on the categorical columns.

</details>



 To select only a subset of features, you can use the `ColumnTransformer` object with the following syntax:
```python
onehot = ColumnTransformer([
    ('name_of_step_1', Transformer_1, list_of_cols_1),
    ('name_of_step_2', Transformer_2, list_of_cols_2),
    ...
    ('name_of_step_n', Transformer_n, list_of_cols_n),
], remainder='passthrough')
```

You can implement a `ColumnTransformer` to only select the categorical features like this:

In [14]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
    ('onehot', OneHotEncoder(drop='first'), categorical_cols)],
    remainder='passthrough')
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(drop='first'),
                                 ['work_type', 'smoking_status', 'who',
                                  'gender', 'residence_type'])])

Running the `.fit_transform()` and `.tranform()` methods, we get:

In [15]:
X_train_encoded = ct.fit_transform(X_train)
X_test_encoded = ct.transform(X_test)

In [16]:
X_train.shape, X_train_encoded.shape

((3831, 11), (3831, 17))

<mark>**Question**:</mark> There were 11 columns and now there are 22. Does that make sense?

In [17]:
ct.get_feature_names_out()

array(['onehot__work_type_Never_worked', 'onehot__work_type_Private',
       'onehot__work_type_Self-employed', 'onehot__work_type_children',
       'onehot__smoking_status_formerly smoked',
       'onehot__smoking_status_never smoked',
       'onehot__smoking_status_smokes', 'onehot__who_Man',
       'onehot__who_Woman', 'onehot__gender_Male',
       'onehot__residence_type_Urban', 'remainder__age',
       'remainder__hypertension', 'remainder__heart_disease',
       'remainder__ever_married', 'remainder__avg_glucose_level',
       'remainder__bmi'], dtype=object)

<details>
<summary><span style="color:blue">Show solution</span></summary>

Using `ct.get_feature_names_out()`, you can see that it created a column for each category: `onehot__gender_Male`, `onehot__gender_Female`, `onehot__work_type_Govt_job`, and so on... There were 17 categories and 5 remaining numerical features that were not one-hot encoded.

</details>

### <mark>Exercise:</mark> Drop-first One-Hot Encoding

It is not necessary to add a column for each categorical value.

There are two options to add to the `drop=` parameter in the `OneHotEncoder`:

- `'first'`
- `'if_binary'`

**Question 1:** What do each of the parameters do? Why would you drop a column?

<details>
  <summary><span style="color:blue">Show solution</span></summary>
    
`drop='first'` will remove one column for each feature

`drop='if_binary'` will only remove one column for the binary features

**Why drop a column in the first place?** Because you actually need one less column than categories to fully encode all the information. E.g., if `gender__Male = 1` you know that `gender__Female` must be zero. So you can drop one column and still have all the information. ML practice follows the *principle of parsimony*. If a simpler model (e.g., fewer features) works as well as a more complex model (e.g., more features), you will prefer the simpler model.


</details>

**Question 2:** Rebuild the `ColumnTransformer` with the correct parameter!

In [18]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

new_ct = ColumnTransformer([
    # your code here
    ('onehot', OneHotEncoder(drop='first'), categorical_cols)],
    remainder='passthrough')
ct

/home/codespace/.local/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(drop='first'),
                                 ['work_type', 'smoking_status', 'who',
                                  'gender', 'residence_type'])])

In [19]:
# %load answers/02-ohe.py

<a id='impute'></a>

### Imputing missing values

Recall that there were two numeric features with missing values. 

In [20]:
missing_cols = list(stroke.columns[stroke.isnull().any()])
missing_cols

['age', 'bmi']

You can use the [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) from sklearn to fill in these missing values.

It will fill the missing values with some constant, e.g., the *median* value of that feature column.

In [21]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='median')

In [22]:
X_train_imputed = impute.fit_transform(X_train_encoded)
X_test_imputed = impute.transform(X_test_encoded)

Applying it directly to the (encoded) data means the same imputing strategy will be used for each column.

In [23]:
X_train['age'].median()

45.0

In [24]:
X_train['bmi'].median()

28.0

However, if you employed the `ColumnTransformer` you could apply different strategies to the columns.

*Note: If the strategy is mean or median, this transformer will only work when all columns are numeric (so you need to impute after the one-hot transformer has been implemented).*

<a id=pipeline></a>

## Sklearn Pipeline

To do this all in one go, you can use the sklearn `Pipeline` object.


<img src="images/sklearn-pipe.png" style="display: block;margin-left: auto;margin-right: auto;width: 400px" align='right'/>

**Pipelines** can encapsulate all the preprocessing steps (feature selections, scaling, encoding of variables and so on), as well as the final model, into a single Scikit-Learn estimator, thereby simplifying and automating many steps.

Pipelines are defined as a **list of steps**, with each step being a `(name, object)` **tuple**:

```Python
pipe = Pipeline(steps=[
    ('name_of_step_1', Transformer/Estimator/Model/Pipeline_1),
    ('name_of_step_2', Transformer/Estimator/Model/Pipeline_2),
    ...
    ('name_of_step_n', Transformer/Estimator/Model/Pipeline_n),
])
```

In [25]:
ct = ColumnTransformer([
    ('onehot', OneHotEncoder(drop="if_binary", sparse_output=False), categorical_cols),
    ], 
    remainder='passthrough', 
    )

#### <mark>Exercise:</mark> Build a `Pipeline`

Build a pipeline called `preprocessing`.
1. In the first step, you should add the `ct` Columntransformer.
2. In the second step, add the `SimpleImputer` with `strategy='mean'`.
3. Check if the data was correctly transformed by using the `.fit_transform(X_train)` method on your preprocessing pipeline. Does the output make sense?
4. Create a new pipeline that adds the Decision Tree Classifier after the preprocessing steps.

*Note: You can output a pandas dataframe by using `preprocessing.set_output(transform='pandas')` before calling the transform method.*

In [26]:
from sklearn.pipeline import Pipeline

# Your code here
impute = SimpleImputer(strategy='median')
ct.SimpleImputer(strategy='mean')
ct.fit_transform(X_train)


AttributeError: 'ColumnTransformer' object has no attribute 'SimpleImputer'

In [ ]:
# %load answers/02-pipeline.py

You can access parts of the pipeline by indexing on their names:

In [ ]:
pipeline['preprocessing']['ct']

This way, you can still access the estimators/transformers/models and their parameters. 

E.g. the feature names after one-hot encoding:

In [ ]:
all_features = pipeline['preprocessing']['ct'].get_feature_names_out()
all_features

### Model creation and evaluation

Let's fit the new pipeline to the original `X_train` and `y_train` data and compare it to the baseline you achieved earlier.

In [ ]:
pipeline.fit(X_train, y_train)

Now, let's look at the ROC-AUC:

In [ ]:
from sklearn.metrics import RocCurveDisplay

fig, ax = plt.subplots(1, 2, figsize=(16,6))

RocCurveDisplay.from_estimator(base_model, X_baseline_train, y_train, ax=ax[0], name='Baseline')
RocCurveDisplay.from_estimator(pipeline, X_train, y_train, ax=ax[0], name='Improved Model')
ax[0].set(title='Train');

RocCurveDisplay.from_estimator(base_model, X_baseline_test, y_test, ax=ax[1], name='Baseline')
RocCurveDisplay.from_estimator(pipeline, X_test, y_test, ax=ax[1], name='Improved Model')
ax[1].set(title='Test')

A good improvement!

Let's see what features the Decision Tree Classifier now used for its splitting rules:

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(20,20))

plot_tree(pipeline.named_steps['model'], 
          feature_names=list(all_features),
          ax=ax);

---

<img src='images/gdd-logo.png' width='300px' align='right' style="padding: 15px">
<a id=conc></a>

# Conclusion and next steps

This notebook covered the main objects in scikit-learn: Estimator, Predictor, Model, Transformer. Two transformers were used to preprocess the data to treat categorical features and features with missing values.

Pipelines were also used as an elegant way to write your code.

Next up you will work to improve this model even further by focussing on the model algorithm.